In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
# import ogb
from tqdm import tqdm
# import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
cwd = os.getcwd()
print(cwd)
# cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
cwd_parent = os.path.abspath(os.path.join(cwd, '../../'))
print(cwd_parent)

sys.path.append(cwd_parent)

/opt/data/chemprop_run/git/notebooks/AltModels
/opt/data/chemprop_run/git


In [4]:
import deepadr
from deepadr.rapidsutilities import *

In [5]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

### Preparing dataset 

In [6]:
score = 'total_thresh'
score_val = 4

In [7]:
DSdataset_name = f'DrugComb_{score}_{score_val}'

# v_1: GNN
# v_2: Alt Models (Baseline)
data_fname = 'data_v2'

In [8]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# print(targetdata_dir)

path_current_dir /opt/data/chemprop_run/git/deepadr
path_current_dir /opt/data/chemprop_run/git/deepadr
path_current_dir /opt/data/chemprop_run/git/deepadr
path_current_dir /opt/data/chemprop_run/git/deepadr


In [9]:
x = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'X_flat_np.pkl'))
y = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'y.pkl'))
# expression = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'expression.pkl'))

In [10]:
y

array([0, 0, 0, ..., 0, 1, 1])

In [11]:
fold_partitions = get_stratified_partitions(y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.5764356969533567
class: 1 norm count: 0.4235643030466433
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5763929334109882
class: 1 norm count: 0.4236070665890118

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.576

In [12]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


Number of training graphs: 18544
Number of validation graphs: 2061
Number of testing graphs: 5152


### Alt Models

In [13]:
# from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.inspection import permutation_importance

### Rapids Models

In [14]:
import cudf
from cuml.linear_model import LogisticRegression as cuLogReg
from cuml.explainer import KernelExplainer, PermutationExplainer

In [15]:
x_np = x
y_np = y

In [16]:
scaler = StandardScaler()
x_np_scale = scaler.fit_transform(x_np)

In [17]:
targetdata_dir_exp

'/opt/data/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v2/experiments'

In [18]:
# model = LogisticRegression(max_iter=10000,
#                                       random_state=42,
#                                       solver="lbfgs",
#                                       penalty='l2',
#                                       l1_ratio=None,
#                                       verbose=1)

In [19]:
model = cuLogReg()

In [20]:
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
print("Start: " + time_stamp)

q_i = 0

# for q_i in range(len(fold_partitions)):
partition = fold_partitions[q_i]

ids_train = partition['train']
ids_test = partition['test']

part_train_x = np.take(x_np_scale, ids_train, axis=0)
part_train_y = np.take(y_np, ids_train, axis=0)

part_test_x = np.take(x_np_scale, ids_test, axis=0)
part_test_y = np.take(y_np, ids_test, axis=0)

cu_train_x = cudf.DataFrame(part_train_x)
cu_train_y = cudf.Series(part_train_y)

cu_test_x = cudf.DataFrame(part_test_x)
cu_test_y = cudf.Series(part_test_y)

exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
create_directory(os.path.join(exp_dir, "predictions"))

model_fit = model.fit(cu_train_x, cu_train_y)

# prob_scores_arr = model_fit.predict_proba(cu_test_x)
# pred_class = model_fit.predict(cu_test_x)
# ref_class = cu_test_y
# epoch = 0
# dsettype = 'test'

# dset_perf = perfmetric_report(pred_class, ref_class, prob_scores_arr[:,1], epoch,
#                           outlog = os.path.join(exp_dir, dsettype + ".log"))

# predictions_df = build_predictions_df(ids_test, ref_class, pred_class, prob_scores_arr)
# predictions_df.to_csv(os.path.join(exp_dir, 'predictions', f'epoch_{epoch}_predictions_{dsettype}.csv'))
    
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

Start: 2022-06-20_17-42-19
path_current_dir /opt/data/chemprop_run/git/deepadr
path_current_dir /opt/data/chemprop_run/git/deepadr
End: 2022-06-20_17-42-21


In [21]:
# time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# print("Start: " + time_stamp)

# q_i = 0

# # for q_i in range(len(fold_partitions)):
# partition = fold_partitions[q_i]

# ids_train = partition['train']
# ids_test = partition['test']

# part_train_x = np.take(x_np_scale, ids_train, axis=0)
# part_train_y = np.take(y_np, ids_train, axis=0)

# part_test_x = np.take(x_np_scale, ids_test, axis=0)
# part_test_y = np.take(y_np, ids_test, axis=0)

# exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
# create_directory(os.path.join(exp_dir, "predictions"))

# model_fit = model.fit(part_train_x, part_train_y)

# prob_scores_arr = model_fit.predict_proba(part_test_x)
# pred_class = model_fit.predict(part_test_x)
# ref_class = part_test_y
# epoch = 0
# dsettype = 'test'

# dset_perf = perfmetric_report(pred_class, ref_class, prob_scores_arr[:,1], epoch,
#                           outlog = os.path.join(exp_dir, dsettype + ".log"))

# predictions_df = build_predictions_df(ids_test, ref_class, pred_class, prob_scores_arr)
# predictions_df.to_csv(os.path.join(exp_dir, 'predictions', f'epoch_{epoch}_predictions_{dsettype}.csv'))
    
# print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

In [22]:
# time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# print("Start: " + time_stamp)

# for q_i in range(len(fold_partitions)):
#     partition = fold_partitions[q_i]
    
#     ids_train = partition['train']
#     ids_test = partition['test']
    
#     part_train_x = np.take(x_np_scale, ids_train, axis=0)
#     part_train_y = np.take(y_np, ids_train, axis=0)
    
#     part_test_x = np.take(x_np_scale, ids_test, axis=0)
#     part_test_y = np.take(y_np, ids_test, axis=0)
    
#     exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
#     create_directory(os.path.join(exp_dir, "predictions"))
    
#     model_fit = model.fit(part_train_x, part_train_y)
    
#     prob_scores_arr = model_fit.predict_proba(part_test_x)
#     pred_class = model_fit.predict(part_test_x)
#     ref_class = part_test_y
#     epoch = 0
#     dsettype = 'test'

#     dset_perf = perfmetric_report(pred_class, ref_class, prob_scores_arr[:,1], epoch,
#                               outlog = os.path.join(exp_dir, dsettype + ".log"))
    
#     predictions_df = build_predictions_df(ids_test, ref_class, pred_class, prob_scores_arr)
#     predictions_df.to_csv(os.path.join(exp_dir, 'predictions', f'epoch_{epoch}_predictions_{dsettype}.csv'))
    
# print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

### SHAP values

In [23]:
# cu_explainer = KernelExplainer(
#     model=model.predict,
#     data=cu_train_x,
#     is_gpu_model=True,
#     random_state=42)

cu_explainer = PermutationExplainer(
    model=model.predict,
    data=cu_train_x,
    is_gpu_model=True,
    random_state=42)

In [24]:
cu_shap_values = cu_explainer.shap_values(cu_test_x)
cu_shap_values  

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /workspace/.conda-bld/work/include/rmm/mr/device/cuda_memory_resource.hpp:70: cudaErrorMemoryAllocation out of memory

### Scores

In [42]:
time_stamp = "2022-06-20_15-29-16"

import glob
exp_dirs = glob.glob(targetdata_dir_exp+"/fold_*_"+time_stamp)
len(exp_dirs)

1

In [35]:
folds = []

for edir in exp_dirs:
    fold = edir.split("/")[-1].split('_')[1]
    with open(os.path.join(edir, "test.log")) as f:
        lines = f.read().splitlines()
        folds.append([fold, float(lines[18]), float(lines[22])])

In [36]:
df_folds = pd.DataFrame(data=folds, columns=["Fold","AUPR", "AUC"]).set_index("Fold")
# df_folds["Fscore"] = df_folds.apply(lambda x: (2*x[0]*x[1])/(x[0]+x[1]), axis=1) # harmonic mean of AUC, AUPR
df_folds

,AUPR,AUC
Fold,,
3,0.828877,0.811873
4,0.842180,0.825904
2,0.830765,0.813108
1,0.836757,0.819945
0,0.835450,0.819092


In [37]:
df_folds.mean(axis=0).values

array([0.8348058 , 0.81798434])

In [38]:
F_score(*df_folds.mean(axis=0).values)

0.8263094695742067